In [17]:
import os.path
import mne
import pandas as pd

from utils.dataloader import load_subject_labels

wd = '/Volumes/Guillaume EEG Project'
epoch_data_path = os.path.join(wd, 'Berlin_Data/EEG/preprocessed/stim_epochs_incl_response')
raw_data_path = os.path.join(wd, 'Berlin_Data/EEG/raw')

In [18]:
# Load the raw data
subject_id = 9

raws = []
for file in [os.path.join(wd, raw_data_path, '{}/{}{}.vhdr'.format(subject_id, subject_id, suffix)) for suffix in ['', '_2', '_3', '_4', '_5']]:
    if os.path.isfile(file):
        raw = mne.io.read_raw_brainvision(file)
        raws.append(raw)

behav_data = load_subject_labels(os.path.join(wd, raw_data_path), subject_id=subject_id)

print(behav_data)

Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/9/9.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/9/9_2.vhdr...
Setting channel info structure...
Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/9/9_3.vhdr...
Setting channel info structure...
{np.int64(4):     response  confidence  correct  session  run
0        NaN         NaN      NaN        1    4
1        1.0         2.0      1.0        1    4
2        1.0         2.0      1.0        1    4
3        1.0         2.0      1.0        1    4
4        NaN         NaN      NaN        1    4
5        1.0         2.0      1.0        1    4
6        1.0         2.0      1.0        1    4
7        1.0         2.0      1.0        1    4
8        1.0         2.0      1.0        1    4
9        1.0         2.0      1.0        1    4
10       1.0         2.0      1.0        1    4
11       1.0         2.0      1.0 

In [19]:
## SPLIT RAW INTO BLOCKS
# Specify the annotation type you want to use for splitting
split_annotation_type = 'Stimulus/S105'

raw_segments = []
for raw in raws:
    # Find the onset times of the relevant annotations
    split_onsets = [annot['onset'] for annot in raw.annotations if annot['description'] == split_annotation_type]

    # Sort the onsets and add the start and end times of the raw object for splitting
    split_onsets = [0] + split_onsets + [raw.times[-1]]

    # Iterate over consecutive pairs of split points to create new raw segments
    for start, end in zip(split_onsets[:-1], split_onsets[1:]):
        raw_segment = raw.copy().crop(tmin=start, tmax=end, include_tmax=False)
        raw_segments.append(raw_segment)

# Now raw_segments is a list of mne.io.Raw objects split at the specified annotations
print(len(raw_segments))

for seg in raw_segments:
    print(seg)
    seg_anno_vc = pd.Series(seg.annotations.description).value_counts()
    print(seg_anno_vc)
    print('')

12
<RawBrainVision | 9.eeg, 63 x 17765 (17.8 s), ~93 kB, data not loaded>
New Segment/     1
Stimulus/S105    1
Name: count, dtype: int64

<RawBrainVision | 9.eeg, 63 x 203467 (203.5 s), ~93 kB, data not loaded>
Stimulus/S 50    151
Stimulus/S150     16
Stimulus/S 64     16
Stimulus/S151     15
Stimulus/S 46     15
Stimulus/S 73     15
Stimulus/S 56     15
Stimulus/S 48     15
Stimulus/S 41     14
Stimulus/S 88      8
Stimulus/S 12      8
Stimulus/S  8      8
Stimulus/S  7      8
Stimulus/S  5      8
Stimulus/S  6      8
Stimulus/S 76      7
Stimulus/S 29      5
Stimulus/S 11      5
Stimulus/S 23      4
Stimulus/S 21      3
Stimulus/S 10      2
Stimulus/S105      2
Stimulus/S 28      2
Stimulus/S 40      2
Stimulus/S  4      1
Name: count, dtype: int64

<RawBrainVision | 9.eeg, 63 x 97562 (97.6 s), ~93 kB, data not loaded>
Stimulus/S 50    91
Stimulus/S150    10
Stimulus/S 64    10
Stimulus/S 56     9
Stimulus/S 73     9
Stimulus/S 48     9
Stimulus/S 46     9
Stimulus/S151     9
Stimu

In [16]:
## EXTRACT EPOCHS
# Step 2: Specify the annotation types you are interested in
annotation_types = ['Stimulus/S{:>3}'.format(stim) for stim in [20, 21, 22, 23]]  # Replace with your specific annotation labels
print(annotation_types)

# Step 3: Convert annotations to events
event_id = {label: idx + 1 for idx, label in enumerate(annotation_types)}  # Create a mapping of annotation labels to event IDs
events, _ = mne.events_from_annotations(raw, event_id=event_id)

# Step 4: Define the epochs
tmin = -2.25  # Start of each epoch (e.g., 200 ms before the event)
tmax = 0.25   # End of each epoch (e.g., 500 ms after the event)

epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)

# Step 5: Inspect the epochs
print(epochs)

['Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 23']
Used Annotations descriptions: [np.str_('Stimulus/S 20'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 23')]
Not setting metadata
193 matching events found
Setting baseline interval to [-2.25, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 193 events and 2501 original time points ...
0 bad epochs dropped
<Epochs | 193 events (all good), -2.25 – 0.25 s (baseline -2.25 – 0 s), ~232.1 MB, data loaded,
 'Stimulus/S 20': 80
 'Stimulus/S 21': 8
 'Stimulus/S 22': 2
 'Stimulus/S 23': 103>
